In [ ]:
import pandas as pd
import numpy as np

This module we'll be looking at the New York City tree census. This data was provided by a volunteer driven census in 2015, and we'll be accessing it via the socrata API. The main site for the data is [here](https://data.cityofnewyork.us/Environment/2015-Street-Tree-Census-Tree-Data/uvpi-gqnh), and on the upper right hand side you'll be able to see the link to the API.

The data is conveniently available in json format, so we should be able to just read it directly in to Pandas:

In [ ]:
url = 'https://data.cityofnewyork.us/resource/nwxe-4ae8.json'
trees = pd.read_json(url)
trees.head(10)

,tree_id,block_id,created_at,tree_dbh,stump_diam,curb_loc,status,health,spc_latin,spc_common,...,boro_ct,state,latitude,longitude,x_sp,y_sp,council_district,census_tract,bin,bbl
0,180683,348711,2015-08-27,3,0,OnCurb,Alive,Fair,Acer rubrum,red maple,...,4073900,New York,40.723092,-73.844215,1.027431e+06,202756.7687,29.0,739.0,4052307.0,4.022210e+09
1,200540,315986,2015-09-03,21,0,OnCurb,Alive,Fair,Quercus palustris,pin oak,...,4097300,New York,40.794111,-73.818679,1.034456e+06,228644.8374,19.0,973.0,4101931.0,4.044750e+09
2,204026,218365,2015-09-05,3,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,3044900,New York,40.717581,-73.936608,1.001823e+06,200716.8913,34.0,449.0,3338310.0,3.028870e+09
3,204337,217969,2015-09-05,10,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,3044900,New York,40.713537,-73.934456,1.002420e+06,199244.2531,34.0,449.0,3338342.0,3.029250e+09
4,189565,223043,2015-08-30,21,0,OnCurb,Alive,Good,Tilia americana,American linden,...,3016500,New York,40.666778,-73.975979,9.909138e+05,182202.4260,39.0,165.0,3025654.0,3.010850e+09
5,190422,106099,2015-08-30,11,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,1014500,New York,40.770046,-73.984950,9.884187e+05,219825.5227,3.0,145.0,1076229.0,1.011310e+09
6,190426,106099,2015-08-30,11,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,1014500,New York,40.770210,-73.985338,9.883112e+05,219885.2785,3.0,145.0,1076229.0,1.011310e+09
7,208649,103940,2015-09-07,9,0,OnCurb,Alive,Good,Tilia americana,American linden,...,1012700,New York,40.762724,-73.987297,9.877691e+05,217157.8561,3.0,133.0,1086093.0,1.010410e+09
8,209610,407443,2015-09-08,6,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,5006400,New York,40.596579,-74.076255,9.630732e+05,156635.5542,NaN,NaN,NaN,NaN
9,192755,207508,2015-08-31,21,0,OffsetFromCurb,Alive,Fair,Platanus x acerifolia,London planetree,...,3037402,New York,40.586357,-73.969744,9.926537e+05,152903.6306,47.0,37402.0,3320727.0,3.072350e+09


Looks good, but lets take a look at the shape of this data:

In [ ]:
trees.shape

(1000, 45)

1000 seems like too few trees for a city like New York, and a suspiciously round number. What's going on?

Socrata places a 1000 row limit on their API. Raw data is meant to be "paged" through for applications, with the expectation that a UX wouldn't be able to handle a full dataset. 

As a simple example, if we had a mobile app with limited space that only displayed trees 5 at a time, we could view the first 5 trees in the dataset with the url below:

In [ ]:
firstfive_url = 'https://data.cityofnewyork.us/resource/nwxe-4ae8.json?$limit=5&$offset=0'
firstfive_trees = pd.read_json(firstfive_url)
firstfive_trees

,tree_id,block_id,created_at,tree_dbh,stump_diam,curb_loc,status,health,spc_latin,spc_common,...,boro_ct,state,latitude,longitude,x_sp,y_sp,council_district,census_tract,bin,bbl
0,180683,348711,2015-08-27,3,0,OnCurb,Alive,Fair,Acer rubrum,red maple,...,4073900,New York,40.723092,-73.844215,1027431.148,202756.7687,29,739,4052307,4022210001
1,200540,315986,2015-09-03,21,0,OnCurb,Alive,Fair,Quercus palustris,pin oak,...,4097300,New York,40.794111,-73.818679,1034455.701,228644.8374,19,973,4101931,4044750045
2,204026,218365,2015-09-05,3,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,3044900,New York,40.717581,-73.936608,1001822.831,200716.8913,34,449,3338310,3028870001
3,204337,217969,2015-09-05,10,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,3044900,New York,40.713537,-73.934456,1002420.358,199244.2531,34,449,3338342,3029250001
4,189565,223043,2015-08-30,21,0,OnCurb,Alive,Good,Tilia americana,American linden,...,3016500,New York,40.666778,-73.975979,990913.775,182202.4260,39,165,3025654,3010850052


If we wanted the next 5, we would use this url:

In [ ]:
nextfive_url = 'https://data.cityofnewyork.us/resource/nwxe-4ae8.json?$limit=5&$offset=5'
nextfive_trees = pd.read_json(nextfive_url)
nextfive_trees

,tree_id,block_id,created_at,tree_dbh,stump_diam,curb_loc,status,health,spc_latin,spc_common,...,boro_ct,state,latitude,longitude,x_sp,y_sp,council_district,census_tract,bin,bbl
0,190422,106099,2015-08-30,11,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,1014500,New York,40.770046,-73.984950,988418.6997,219825.5227,3.0,145.0,1076229.0,1.011310e+09
1,190426,106099,2015-08-30,11,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,1014500,New York,40.770210,-73.985338,988311.1900,219885.2785,3.0,145.0,1076229.0,1.011310e+09
2,208649,103940,2015-09-07,9,0,OnCurb,Alive,Good,Tilia americana,American linden,...,1012700,New York,40.762724,-73.987297,987769.1163,217157.8561,3.0,133.0,1086093.0,1.010410e+09
3,209610,407443,2015-09-08,6,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,5006400,New York,40.596579,-74.076255,963073.1998,156635.5542,NaN,NaN,NaN,NaN
4,192755,207508,2015-08-31,21,0,OffsetFromCurb,Alive,Fair,Platanus x acerifolia,London planetree,...,3037402,New York,40.586357,-73.969744,992653.7253,152903.6306,47.0,37402.0,3320727.0,3.072350e+09


You can read more about paging using the Socrata API [here](https://dev.socrata.com/docs/paging.html)

In these docs, you'll also see more advanced functions (called `SoQL`) under the "filtering and query" section. These functions should be reminding you of SQL.

Think about the shape you want your data to be in before querying it. Using `SoQL` is a good way to avoid the limits of the API. For example, using the below query I can easily obtain the count of each species of tree in the Bronx:

In [ ]:
boro = 'Bronx'
soql_url = ('https://data.cityofnewyork.us/resource/nwxe-4ae8.json?' +\
        '$select=spc_common,count(tree_id)' +\
        '&$where=boroname=\'Bronx\'' +\
        '&$group=spc_common').replace(' ', '%20')
soql_trees = pd.read_json(soql_url)

soql_trees

,count_tree_id,spc_common
0,4619,NaN
1,43,black walnut
2,9,spruce
3,62,tulip-poplar
4,7,trident maple
...,...,...
128,195,crimson king maple
129,774,purple-leaf plum
130,363,sawtooth oak
131,505,willow oak


This behavior is very common with web APIs, and I think this is useful when thinking about building interactive data products. When in a Jupyter Notebook or RStudio, there's an expectation that (unless you're dealing with truly large datasets) the data you want can be brought in memory and manipulated.

Dash and Shiny abstract away the need to distinguish between client side and server side to make web development more accessible to data scientists. This can lead to some unintentional design mistakes if you don't think about how costly your callback functions are (for example: nothing will stop you in dash from running a costly model triggered whenever a dropdown is called.)

The goal of using the Socrata is to force you to think about where your data operations are happening, and not resort to pulling in the data and performing all operations in local memory.

----------

**NOTE**: One tip in dealing with URLs: you may need to replace spaces with `'%20'`. I personally just write out the url and then follow the string with a replace:

In [ ]:
'https://api-url.com/?query with spaces'.replace(' ', '%20')

'https://api-url.com/?query%20with%20spaces'

In [ ]:
#install Dash
!pip install jupyter-dash

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 57.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 62.8 MB/s eta 0:00:00


In [ ]:
from dash import Dash, html, dcc
import plotly.express as px

In [ ]:
soql_url_assignment4 = ('https://data.cityofnewyork.us/resource/nwxe-4ae8.json?' +\
        '$select=health,count(tree_id)' +\
        '&$group=health').replace(' ', '%20')
soql_trees_assignment4 = pd.read_json(soql_url_assignment4)

soql_trees_assignment4

,health,count_tree_id
0,Fair,96504
1,Good,528850
2,Poor,26818
3,NaN,31616


In [ ]:
soql_url_assignment4 = ('https://data.cityofnewyork.us/resource/nwxe-4ae8.json?' +\
        '$select=health,spc_common,boroname,steward').replace(' ', '%20')
soql_trees_assignment4 = pd.read_json(soql_url_assignment4)

soql_trees_assignment4['health']=soql_trees_assignment4.health.fillna('Unknown')


In [ ]:
print(soql_trees_assignment4)

      health       spc_common       boroname steward
0       Fair        red maple         Queens    None
1       Fair          pin oak         Queens    None
2       Good      honeylocust       Brooklyn    1or2
3       Good      honeylocust       Brooklyn    None
4       Good  American linden       Brooklyn    None
..       ...              ...            ...     ...
995  Unknown              NaN  Staten Island     NaN
996  Unknown              NaN  Staten Island     NaN
997     Poor     Norway maple       Brooklyn    1or2
998     Good     Norway maple       Brooklyn    None
999     Good     Norway maple  Staten Island    None

[1000 rows x 4 columns]


In [ ]:
soql_trees_assignment4['steward'].unique()

array(['None', '1or2', '3or4', nan, '4orMore'], dtype=object)

In [ ]:
!pip uninstall dash
!pip install dash

Found existing installation: dash 2.9.1
Uninstalling dash-2.9.1:
  Would remove:
    /usr/local/bin/dash-generate-components
    /usr/local/bin/dash-update-components
    /usr/local/bin/renderer
    /usr/local/lib/python3.9/dist-packages/dash-2.9.1.dist-info/*
    /usr/local/lib/python3.9/dist-packages/dash/*
Proceed (Y/n)? Y
  Successfully uninstalled dash-2.9.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached dash-2.9.1-py3-none-any.whl (10.2 MB)


In [ ]:
!pip install jupyter-dash

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## 1. What proportion of trees are in good, fair, or poor health according to the ‘health’ variable ?

In [59]:
import pandas as pd
import numpy as np
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
from jupyter_dash import JupyterDash
from dash.dependencies import Input, Output
from dash import dash_table, Input, Output

soql_url_assignment4 = ('https://data.cityofnewyork.us/resource/nwxe-4ae8.json?' +\
        '$select=health,spc_common,boroname,steward').replace(' ', '%20')
soql_trees_assignment4 = pd.read_json(soql_url_assignment4)

soql_trees_assignment4=soql_trees_assignment4.dropna()
soql_trees_assignment4_b = soql_trees_assignment4.groupby(["health","boroname","spc_common"])["health"].count().reset_index(name="count")
soql_trees_assignment4_b['Percentage'] = 100 * soql_trees_assignment4_b['count']  / soql_trees_assignment4_b['count'].sum()
soql_trees_assignment4_b

#dash
app = JupyterDash(__name__)
#fig = px.histogram(soql_trees_assignment4, x="health", color="spc_common", barmode="group")
app.layout = html.Div(children=[
    html.H1(children='Brian Singh_Assignment4_Data608_Question1'),

    html.Div(children='''
        Please select a tree species.
    '''),
        
        
        dcc.Dropdown(
            soql_trees_assignment4['spc_common'].unique(),
            id='spc',
            value=soql_trees_assignment4.spc_common.values,
        ),

        dcc.Dropdown(
            soql_trees_assignment4['boroname'].unique(),
            id='boro',
            value=soql_trees_assignment4.boroname.values,
        ),

    dcc.Graph(
        id='graph',
        figure=fig
    ),

    dash_table.DataTable(id="table")
    #dash_table.DataTable(id="table",
     #                   columns=[{'name':i,"id":i} for i in soql_trees_assignment4_b.columns],
      #                   data= soql_trees_assignment4_b.to_dict('records')),

    #dcc.Graph(figure = px.histogram(soql_trees_assignment4, x="health", color="spc_common", barmode="group"))
])

@app.callback(
    Output('graph','figure'),
    #Output('table','data'), #new
    Input('spc','value'),
    Input('boro','value')
)

def update_graph(spc,boro):
  df = soql_trees_assignment4
  fig = px.histogram(df, x=df["health"].loc[(df["spc_common"]==spc) & (df["boroname"]==boro)])
  return fig

@app.callback(
    Output('table','data'), #new
    Input('spc','value'),
    Input('boro','value')
)

def update_table(spc,boro):
  #df2 = soql_trees_assignment4
  tbl = soql_trees_assignment4_b[(soql_trees_assignment4_b.spc_common.isin(spc)) & (soql_trees_assignment4_b.boroname.isin(boro))]
  #tbl = dash_table.DataTable(columns=[{'name':i,"id":i} for i in df2.loc[(df2["spc_common"]==spc) & (df2["boroname"]==boro)].columns],
  #                       data= df2.to_dict('records'))
  return tbl.to_dict('records')

#if __name__ == '__main__':
app.run_server(mode='inline')

Dash is running on http://127.0.0.1:8050/



INFO:dash.dash:Dash is running on http://127.0.0.1:8050/



<IPython.core.display.Javascript object>

I was having trouble in the code above having the Dash Table update with the dropdown filters as the graph updated. I saw this code on https://dash.plotly.com/datatable/callbacks and implemented using this dataset. It isn't formatted ideally, but it gets me in the direction I was aiming for. The table allows for filtering and updates the graphs accordingly.

In [58]:
#working2

import pandas as pd
import numpy as np
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
from jupyter_dash import JupyterDash
from dash.dependencies import Input, Output
from dash import dash_table, Input, Output

soql_url_assignment4 = ('https://data.cityofnewyork.us/resource/nwxe-4ae8.json?' +\
        '$select=health,spc_common,boroname,steward').replace(' ', '%20')
soql_trees_assignment4 = pd.read_json(soql_url_assignment4)

soql_trees_assignment4=soql_trees_assignment4.dropna()
soql_trees_assignment4_b = soql_trees_assignment4.groupby(["health","boroname","spc_common"])["health"].count().reset_index(name="count")
soql_trees_assignment4_b['Percentage'] = 100 * soql_trees_assignment4_b['count']  / soql_trees_assignment4_b['count'].sum()
soql_trees_assignment4_b

#dash
app = JupyterDash(__name__)
#fig = px.histogram(soql_trees_assignment4, x="health", color="spc_common", barmode="group")
app.layout = html.Div(
    className="row",
    children=[
        html.Div(
            dash_table.DataTable(
                id='table-paging-with-graph',
                columns=[
                    {"name": i, "id": i} for i in sorted(soql_trees_assignment4_b.columns)
                ],
                page_current=0,
                page_size=20,
                page_action='custom',

                filter_action='custom',
                filter_query='',

                sort_action='custom',
                sort_mode='multi',
                sort_by=[]
            ),
            style={'height': 750, 'overflowY': 'scroll'},
            className='six columns'
        ),
        html.Div(
            id='table-paging-with-graph-container',
            className="five columns"
        )
    ]
)

operators = [['ge ', '>='],
             ['le ', '<='],
             ['lt ', '<'],
             ['gt ', '>'],
             ['ne ', '!='],
             ['eq ', '='],
             ['contains '],
             ['datestartswith ']]


def split_filter_part(filter_part):
    for operator_type in operators:
        for operator in operator_type:
            if operator in filter_part:
                name_part, value_part = filter_part.split(operator, 1)
                name = name_part[name_part.find('{') + 1: name_part.rfind('}')]

                value_part = value_part.strip()
                v0 = value_part[0]
                if (v0 == value_part[-1] and v0 in ("'", '"', '`')):
                    value = value_part[1: -1].replace('\\' + v0, v0)
                else:
                    try:
                        value = float(value_part)
                    except ValueError:
                        value = value_part

                # word operators need spaces after them in the filter string,
                # but we don't want these later
                return name, operator_type[0].strip(), value

    return [None] * 3


@app.callback(
    Output('table-paging-with-graph', "data"),
    Input('table-paging-with-graph', "page_current"),
    Input('table-paging-with-graph', "page_size"),
    Input('table-paging-with-graph', "sort_by"),
    Input('table-paging-with-graph', "filter_query"))
def update_table(page_current, page_size, sort_by, filter):
    filtering_expressions = filter.split(' && ')
    dff = soql_trees_assignment4_b
    for filter_part in filtering_expressions:
        col_name, operator, filter_value = split_filter_part(filter_part)

        if operator in ('eq', 'ne', 'lt', 'le', 'gt', 'ge'):
            # these operators match pandas series operator method names
            dff = dff.loc[getattr(dff[col_name], operator)(filter_value)]
        elif operator == 'contains':
            dff = dff.loc[dff[col_name].str.contains(filter_value)]
        elif operator == 'datestartswith':
            # this is a simplification of the front-end filtering logic,
            # only works with complete fields in standard format
            dff = dff.loc[dff[col_name].str.startswith(filter_value)]

    if len(sort_by):
        dff = dff.sort_values(
            [col['column_id'] for col in sort_by],
            ascending=[
                col['direction'] == 'asc'
                for col in sort_by
            ],
            inplace=False
        )

    return dff.iloc[
        page_current*page_size: (page_current + 1)*page_size
    ].to_dict('records')


@app.callback(
    Output('table-paging-with-graph-container', "children"),
    Input('table-paging-with-graph', "data"))
def update_graph(rows):
    dff = pd.DataFrame(rows)
    return html.Div(
        [
            dcc.Graph(
                id=column,
                figure={
                    "data": [
                        {
                            "x": dff["health"],
                            "y": dff[column] if column in dff else [],
                            "type": "bar",
                            "marker": {"color": "#0074D9"},
                        }
                    ],
                    "layout": {
                        "xaxis": {"automargin": True},
                        "yaxis": {"automargin": True},
                        "height": 250,
                        "margin": {"t": 10, "l": 10, "r": 10},
                    },
                },
            )
            for column in ["count", "Percentage"]
        ]
    )

#if __name__ == '__main__':
app.run_server(mode='inline')

Dash is running on http://127.0.0.1:8050/



INFO:dash.dash:Dash is running on http://127.0.0.1:8050/



<IPython.core.display.Javascript object>

##2. Are stewards (steward activity measured by the ‘steward’ variable) having an impact on the health of trees?

In [65]:
app2 = JupyterDash(__name__)
#fig = px.histogram(soql_trees_assignment4, x="health", color="spc_common", barmode="group")
app2.layout = html.Div(children=[
    html.H1(children='Brian Singh_Assignment4_Data608_Question2'),

    html.Div(children='''
        Please select a steward category.
    '''),

        
        dcc.Dropdown(
            soql_trees_assignment4['steward'].unique(),
            id='stwd',
            value=soql_trees_assignment4.steward.values,
        ),

        dcc.Dropdown(
            soql_trees_assignment4['spc_common'].unique(),
            id='spc',
            value=soql_trees_assignment4.spc_common.values,
        ),

        dcc.Dropdown(
            soql_trees_assignment4['boroname'].unique(),
            id='boro',
            value=soql_trees_assignment4.boroname.values,
        ),

    dcc.Graph(
        id='graph',
        figure=fig
    )
])

@app2.callback(
    Output('graph','figure'),
    #Output('table','data'), #new
    Input('stwd','value'),
    Input('spc','value'),
    Input('boro','value')
)

def update_graph(stwd,spc,boro):
  df3 = soql_trees_assignment4
  fig = px.histogram(df3, x=df3["health"].loc[(df3["steward"]==stwd) & (df3["spc_common"]==spc) & (df3["boroname"]==boro)])
  return fig

app2.run_server(mode='inline')

Dash is running on http://127.0.0.1:8050/



INFO:dash.dash:Dash is running on http://127.0.0.1:8050/



<IPython.core.display.Javascript object>

## Conclusion

In the first set of code for #1, I was able to successfully implement dropdowns for the tree species and borough in order for the user to visually see the tree's health. I was unable to get the table to update with the same dropdown filters, however in code I found via the link referenced above for my second set of code, I was able to output the table of data and the ability to filter also filters the histograms.

For question #2, we can see that trees with no stewards have the most trees in good health, which is interesting. However, this could be due to the fact that a majority of trees do not have a steward assigned.